In [ ]:
import pandas as pd
import numpy as np
import itertools
from sklearn import preprocessing

In [ ]:
pathway_geneName_ensembl=pd.read_csv('data/pathways_with_ensembl_norowname.csv', engine='python', header=0, index_col=None)

In [ ]:
pathway_geneName_ensembl.head()

In [ ]:
%%time
#读取基因表达数据
pan_gene_filtered = pd.read_hdf("data/Preprocessed_Data/pan_gene_filtered_1k.h5", 
                      key = "pan_gene_filtered")
lung_gene_filtered = pd.read_hdf("data/Preprocessed_Data/lung_gene_filtered_1k.h5", 
                      key = "lung_gene_filtered")

In [ ]:
for i in list(pan_gene_filtered.columns):
    Max = np.max(pan_gene_filtered[i])
    Min = np.min(pan_gene_filtered[i])
    pan_gene_filtered[i] = (pan_gene_filtered[i] - Min)/(Max - Min)

In [ ]:
pan_gene_filtered=pan_gene_filtered.fillna(0)

In [ ]:
pan_index=[]
for ensembl in pan_gene_filtered.T.index:
    pan_index.append(ensembl[0:15])

In [ ]:
pan_gene_filtered.columns=pan_index

In [ ]:
pan_gene_filtered

In [ ]:
for i in list(lung_gene_filtered.columns):
   # 获取各个指标的最大值和最小值
    Max = np.max(lung_gene_filtered[i])
    Min = np.min(lung_gene_filtered[i])
    lung_gene_filtered[i] = (lung_gene_filtered[i] - Min)/(Max - Min)

In [ ]:
lung_gene_filtered=lung_gene_filtered.fillna(0)

In [ ]:
lung_gene_filtered.T.index

In [ ]:
lung_index=[]
for ensembl in lung_gene_filtered.T.index:
    lung_index.append(ensembl[0:15])
lung_gene_filtered.columns=lung_index

In [ ]:
lung_gene_filtered

In [ ]:
#number of ensembl geneids associated with pathways
len(set(pathway_geneName_ensembl['ensembl']))

In [ ]:
ensembls_expre=pan_gene_filtered.columns

In [ ]:
ensembls_expre

In [ ]:
pathway_geneName_ensembl=pathway_geneName_ensembl[pathway_geneName_ensembl['ensembl'].isin(ensembls_expre)]
pathway_geneName_ensembl.shape

In [ ]:
pathway_geneName_ensembl.head()

In [ ]:
pathway_geneName_ensembl.index=pathway_geneName_ensembl['ensembl']
pathway_geneName_ensembl['ensembl'].name='ensid'

In [ ]:
pan_expre_pathway=pan_gene_filtered.T
pan_expre_pathway.index.name='ensid'
pan_expre_pathway['order']=pan_gene_filtered.mean(axis=0)
pan_expre_pathway=pd.merge(pan_expre_pathway,pathway_geneName_ensembl,on='ensid')

In [ ]:
pan_expre_pathway

In [ ]:
lung_expre_pathway=lung_gene_filtered.T
lung_expre_pathway.index.name='ensid'
lung_expre_pathway['order']=lung_gene_filtered.mean(axis=0)
lung_expre_pathway=pd.merge(lung_expre_pathway,pathway_geneName_ensembl,on='ensid')

In [ ]:
lung_expre_pathway

In [ ]:
%%time
# Save dataset
with pd.HDFStore('data/KEGG/KEGG_pathway_map_log2tpm_MinMax_1k.h5','w') as store:
    store['pan_pathway_map']=pan_expre_pathway
    #store['non_lung_tree_map']=non_lung_tree_map
    store['lung_pathway_map']=lung_expre_pathway